In [2]:
import torch
print(torch.__version__)

1.12.1


In [3]:
# create sample node feature
X = torch.Tensor(torch.randn(6,3)) # torch.randn(#_of_node_feature, feature_dim)

In [4]:
# define sample adjacency matrix
adj = torch.Tensor([[0,1,0,0,1,0],
                    [1,0,1,0,1,0],
                    [0,1,0,1,0,0],
                    [0,0,1,0,1,1],
                    [1,1,0,1,0,0],
                    [0,0,0,1,0,0]])

In [5]:
print(adj)
print(adj.shape)

tensor([[0., 1., 0., 0., 1., 0.],
        [1., 0., 1., 0., 1., 0.],
        [0., 1., 0., 1., 0., 0.],
        [0., 0., 1., 0., 1., 1.],
        [1., 1., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.]])
torch.Size([6, 6])


In [6]:
# need self loops to use self informations
tilde = adj + torch.eye(adj.shape[0])
print(tilde)

tensor([[1., 1., 0., 0., 1., 0.],
        [1., 1., 1., 0., 1., 0.],
        [0., 1., 1., 1., 0., 0.],
        [0., 0., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1., 0.],
        [0., 0., 0., 1., 0., 1.]])


In [7]:
# to sparse matrix
sps = tilde.to_sparse()
print(sps.indices())
print(sps.values())
print(sps.size())

tensor([[0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5],
        [0, 1, 4, 0, 1, 2, 4, 1, 2, 3, 2, 3, 4, 5, 0, 1, 3, 4, 3, 5]])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.])
torch.Size([6, 6])


$Z = \tilde{D}^{-{1\over2}} \tilde{A} \tilde{D}^{-{1\over2}} X\theta$

위 식에서 normalize 부분($\tilde{D}^{-{1\over2}}$)은 잠시 제외하고 계산을 해보자. ($X\theta$도 여기서는 초기화된 행렬 X로 가정)

In [8]:
print(X)
print(tilde)

tensor([[-1.4045, -0.7183,  0.4900],
        [-0.7149, -3.0254, -0.7417],
        [-0.9960, -1.0151, -1.3487],
        [-1.2980,  3.3590, -0.8335],
        [ 0.3266,  1.1231,  0.9904],
        [-1.6103,  0.4507, -0.0049]])
tensor([[1., 1., 0., 0., 1., 0.],
        [1., 1., 1., 0., 1., 0.],
        [0., 1., 1., 1., 0., 0.],
        [0., 0., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1., 0.],
        [0., 0., 0., 1., 0., 1.]])


In [9]:
Z = torch.matmul(tilde, X)
print(Z)

tensor([[-1.7927, -2.6206,  0.7387],
        [-2.7887, -3.6357, -0.6101],
        [-3.0089, -0.6816, -2.9239],
        [-3.5776,  3.9176, -1.1968],
        [-3.0908,  0.7384, -0.0948],
        [-2.9083,  3.8096, -0.8384]])


In [10]:
# Normalize term D
D1 = tilde.sum(dim=0)
D2 = tilde.sum(dim=1)
print(D1)
print(D2)

tensor([3., 4., 3., 4., 4., 2.])
tensor([3., 4., 3., 4., 4., 2.])


Degree의 개수로 normalize를 시켜주는 작업으로, directed graph에서 indegree와 outdegree가 다를 때도 normalize가 가능하다.

$Z = \tilde{D1}^{-{1\over2}} \tilde{A} \tilde{D2}^{-{1\over2}} X\theta$


In [16]:
normalized_tilde = ((D1**(0.5)*tilde)*(D2**(0.5)))
normalized_Z = torch.matmul(normalized_tilde, X)
print(normalized_Z)

tensor([[ -5.7665,  -9.7640,   2.4646],
        [ -8.7544, -12.8094,  -1.5816],
        [-11.0398,  -1.7112, -10.3470],
        [-10.0940,  15.7843,  -3.4286],
        [-10.9587,   3.6719,  -0.8693],
        [ -8.4127,  14.3372,  -3.3438]])
